In [16]:
from brian2 import *
%matplotlib inline

#basic neuronal group characteristics
neurons_num = 12 #referring to the number of stuctures and subgroups sizes

v_rest = -70*mV #resting potential
v_th = -50*mV #threshold potential
area = 0.785*umetre**2
Cm = 1*ufarad*cm**-2 * area
v_eK = -90*mV
v_eNa = 50*mV
g_Na = 100*msiemens*cm**-2 * area
g_K = 30*msiemens*cm**-2 * area
VT = -63*mV


#Hodgkin-Huxley model equations
eqs = '''
dv/dt = -(g_Na*(m*m*m)*h*(v-v_eNa) + g_K*(n*n*n*n)*(v-v_eK) + I_stim)/Cm : volt
dm/dt = 0.32*(mV**-1)*4*mV/exprel((13.*mV-v+VT)/(4*mV))/ms*(1-m)-0.28*(mV**-1)*5*mV/exprel((v-VT-40.*mV)/(5*mV))/ms*m : 1
dn/dt = 0.032*(mV**-1)*5*mV/exprel((15.*mV-v+VT)/(5*mV))/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
I_stim : amp

'''

#glutamate
model_eqs_glu = '''
receptors_num : 1
v_e : volt
W : metre**3
Kd : mM
gam : siemens
dL/dt = -1.1*10**-9 : mol (event-driven)
dI_stim_syn/dt = receptors_num*(v - v_e)*gam*((L/W/Kd)/(1 + (L/W/Kd)))/ms : amp (clock-driven)

'''

#gaba
model_eqs_gaba = '''
receptors_num : 1
v_e : volt
W : metre**3
Kd : mM
gam : siemens
dL/dt = -4*10**-9 : mol (event-driven)
dI_stim_syn/dt = receptors_num*(v - v_e)*gam*((L/W/Kd)/(1 + (L/W/Kd)))/ms : amp (clock-driven)

'''

Prototype = NeuronGroup(neurons_num, eqs, threshold='v > v_th', reset='v = v_rest')
Prototype.v = v_rest

# Mapping subgroups by cerebellum integration circuit
MC_in = Prototype[0]
DSGo = Prototype[1]
DSNoGo = Prototype[2]
GPi = Prototype[3]
GPe = Prototype[5]
STN = Prototype[6]
Th = Prototype[4]
MC_out = Prototype[11]

Pons = Prototype[7]
CC = Prototype[8]
DN = Prototype[9]

SNc = Prototype[10]
#Glu = Prototype[1, 2, 4, 7, 8, 11]
#GABA = Prototype[3, 4, 5, 9]

#Setting up synapses
#excitatory synapses (glutamate)
S_exc = Synapses(Prototype, Prototype, model = Equations(model_eqs_glu), on_pre='L = 1.1*mM')


S_exc.connect(i=MC_in, j=[1, 2])
S_exc.connect(i=Th, j=MC_out)
#S_exc.connect(i=SNc, j=DSGo)

#cerebellum route
S_exc.connect(i=MC_in, j=Pons)
S_exc.connect(i=Pons, j=CC)
S_exc.connect(i=DN, j=Th)

#synapses parameters
S_exc.receptors_num = 2500
S_exc.v_e = +55*mV
S_exc.gam = 10*psiemens
S_exc.W = 20*nmeter * area
S_exc.Kd = 500*nM


#inhibitory synapses (GABA)
S_inh = Synapses(Prototype, Prototype, model = Equations(model_eqs_gaba), on_pre='L = 4*mM')
                
S_inh.connect(i=DSGo, j=GPi)
S_inh.connect(i=DSNoGo, j=GPe)
S_inh.connect(i=GPe, j=GPi)
S_inh.connect(i=GPi, j=Th)
#S_inh.connect(i=SNc, j=DSNoGo)

#cerebellum route
S_inh.connect(i=CC, j=DN)

#synapses parameters
S_inh.receptors_num = 2500
S_inh.v_e = -65*mV
S_inh.gam = 8*psiemens
S_inh.W = 20*nmeter * area
S_inh.Kd = 128*nM


In [17]:
M = StateMonitor(Prototype, 'v', record=True)

run(5000*ms)

plot(M.t/ns, M.v[4], label='Th')

xlabel('Time (ms)')
ylabel('v')
legend();

INFO       No numerical integration method specified for group 'neurongroup_2', using method 'euler' (took 0.01s, trying other methods took 0.06s). [brian2.stateupdaters.base.method_choice]
INFO       No numerical integration method specified for group 'synapses_4', using method 'euler' (took 0.01s, trying other methods took 0.01s). [brian2.stateupdaters.base.method_choice]
INFO       No numerical integration method specified for group 'synapses_5', using method 'euler' (took 0.01s, trying other methods took 0.01s). [brian2.stateupdaters.base.method_choice]


BrianObjectException: Original error and traceback:
Traceback (most recent call last):
  File "/Users/victoria.stelmakh/opt/anaconda3/lib/python3.8/site-packages/brian2/core/network.py", line 901, in before_run
    obj.before_run(run_namespace)
  File "/Users/victoria.stelmakh/opt/anaconda3/lib/python3.8/site-packages/brian2/core/base.py", line 280, in device_override_decorated_function
    return func(*args, **kwds)
  File "/Users/victoria.stelmakh/opt/anaconda3/lib/python3.8/site-packages/brian2/synapses/synapses.py", line 319, in before_run
    CodeRunner.before_run(self, run_namespace)
  File "/Users/victoria.stelmakh/opt/anaconda3/lib/python3.8/site-packages/brian2/groups/group.py", line 1122, in before_run
    self.codeobj = create_runner_codeobj(group=self.group,
  File "/Users/victoria.stelmakh/opt/anaconda3/lib/python3.8/site-packages/brian2/codegen/codeobject.py", line 362, in create_runner_codeobj
    check_units_statements(c, variables)
  File "/Users/victoria.stelmakh/opt/

In [ ]:
#trash
#dn/dt = ((0.01*(10 + v))/(exp((10 + v)/10) - 1))*(1-n) - (0.125*exp(v/80))*n : 1
#dm/dt = ((0.1*(25 + v))/(exp((25 + v)/10) - 1))*(1-m) - (4*exp(v/18))*m : 1
#dh/dt = 0.07*exp(v/20)*(1-h) - (1/(exp((30 + v)/10) + 1))*h : 1